In [ ]:
%load_ext autoreload
%autoreload 2
import cv2
import json
import numpy as np
import matplotlib.pyplot as plt
import random, os, time, copy, glob
import imgaug as ia
import imgaug.augmenters as iaa
from imgaug.augmentables import Keypoint, KeypointsOnImage

#from __future__ import print_function
#from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)
from dataset import TrainCouplingDataset


In [ ]:
batch_size=8


In [ ]:
def opencv_loader(path: str) -> np.array:
    img = cv2.imread(path)
    return img

class TrainCouplingDataset(Dataset):
    
    def __init__(self, filename_list, root_dir, transform=None, mode='val'):
        self.filename_list = filename_list
        self.root_dir = root_dir
        self.transform = transform
        self.mode=mode
        self.train_seq = iaa.Sequential([
            iaa.Resize(448),
            iaa.Sometimes(
                0.2,
                iaa.MotionBlur(k=(5, 15)),
            ),
            iaa.Add((-40, 40)),
            iaa.Multiply((1.2, 1.5)), # change brightness, doesn't affect keypoints
            iaa.Fliplr(0.5),
            iaa.Sometimes(
                0.2,
                iaa.GaussianBlur(sigma=(1.0, 5.0)),
            ),
            
        ])
        self.val_seq = iaa.Sequential([
            iaa.Resize(448),
            
        ])
        

    def __len__(self):
        return len(self.filename_list)

    def __getitem__(self, idx):
        try:
            if torch.is_tensor(idx):
                idx = idx.tolist()

            img_fname = os.path.join(self.root_dir,
                                    os.path.basename(self.filename_list[idx]))
            image = opencv_loader(img_fname)
            #print(img_fname)

            json_fname = '.'.join(img_fname.split('.')[:-1])+'.json'
            j = json.load(open(json_fname))
            pts = j['shapes'][0]['points']
            #print(pts)
            if self.mode == 'train':
                seq = self.train_seq
            else:
                seq = self.val_seq
            
            kps = KeypointsOnImage([
                Keypoint(x=pt[0], y=pt[1]) for pt in pts], 
                shape = image.shape
            )
            #print(kps)
            image_aug, kps_aug = seq(image=image, keypoints=kps)
            image = image_aug
            #print(kps_aug)
            minx =  min([pt.x for pt in kps_aug])
            maxx = max([pt.x for pt in kps_aug])
            
            
            target = [minx/image.shape[1], maxx/image.shape[1]]
            #print(target)
            
            sample = {'image': torch.tensor(image).permute(2, 0, 1).type(torch.FloatTensor), 'bbox': torch.tensor(target).type(torch.FloatTensor)}
            return sample
        except Exception as e:
            print(e)
            raise

In [ ]:
train_filelist = glob.glob('../eyenSE/car_coupling_train/*.jpg')
val_filelist = glob.glob('../eyenSE/car_coupling_train/*.jpeg')
print(val_filelist[0])


In [ ]:
train_dset = TrainCouplingDataset(filename_list = train_filelist, root_dir='../car_coupling_train/', mode='train')
val_dset = TrainCouplingDataset(filename_list = val_filelist, root_dir='../car_coupling_train/', mode='val')
image_datasets = {
    'train': train_dset,
    'val': val_dset,
}
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=0) for x in ['train', 'val']}

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
_ = train_dset[1]
_img = np.array(_['image'].permute(1, 2, 0).numpy(), dtype=np.uint8)
plt.figure(figsize=(20, 20))
plt.imshow(_img)

'''
print(_['bbox'])
print(_['image'].shape)
print(_['image'].type())
print(_)


_ = val_dset[1]
print(_['bbox'])
print(_['image'].shape)
print(_['image'].type())
print(_)
'''


In [ ]:
use_pretrained = True
feature_extract = False
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False
model_ft = models.resnet18(pretrained=True)
#set_parameter_requires_grad(model_ft, feature_extract)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 2)
model_ft.to(device)

In [ ]:
params_to_update = model_ft.parameters()
#print("Params to learn:")
'''
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
'''
for name,param in model_ft.named_parameters():
    if param.requires_grad == True:
        print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 15000

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data
            _dl_start = time.time()
            for x in dataloaders[phase]:
                _dl_end =time.time()
                #print('dataloaders: {:.2f}'.format(_dl_end - _dl_start))
                inputs = x['image']
                inputs = inputs.to(device)
                #print(inputs.shape)
                labels = x['bbox']
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        _fwd_start = time.time()
                        #print(inputs.type())
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)
                        _fwd_end = time.time()
                        #print('forward: {:.2f}'.format(_fwd_end - _fwd_start))

                    #_, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        _back_start = time.time()
                        loss.backward()
                        optimizer.step()
                        _back_end = time.time()
                        #print('backward: {:.2f}'.format(_back_end - _back_start))
                        

                # statistics
                running_loss += loss.item() * inputs.size(0)
                #running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            #epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f}'.format(phase, epoch_loss))

            # deep copy the model
            if phase == 'val' and epoch_loss < best_loss:
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
                
            if phase == 'val':
                val_acc_history.append(epoch_loss)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [ ]:
criterion = nn.MSELoss()

In [ ]:
num_epochs = 100

In [ ]:

model_name='resnet'
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=(model_name=="inception"))

In [ ]:
se = 0.
ctr = 0
torch.save(model_ft.state_dict(), 'model_x2.pth')
for idx in range(len(val_dset)):

    res = model_ft(val_dset[idx]['image'].unsqueeze(0).to(device))[0].detach().cpu().numpy()
    print(res)
    gold = val_dset[idx]['bbox'].numpy() 
    print(gold)
    
    
    
    #se += (res - gold)**2 
    ctr += 1
    print()
print(se/ctr)
print(ctr)

In [ ]:
model_loaded = models.resnet18(pretrained=True)
#set_parameter_requires_grad(model_ft, feature_extract)
num_ftrs = model_loaded.fc.in_features
model_loaded.fc = nn.Linear(num_ftrs, 2)
if device == 'cpu':
    model_loaded.load_state_dict(torch.load('model_x2.pth'))
else:
    model_loaded.load_state_dict(torch.load('model_x2.pth', map_location=torch.device('cpu')))
model_loaded.eval()
model_loaded.to(device)

#odel_loaded.to(device)


for idx in range(len(val_dset)):

    res = model_loaded(val_dset[idx]['image'].unsqueeze(0).to(device))[0].detach().cpu().numpy()
    print(res)
    gold = val_dset[idx]['bbox'].numpy() 
    print(gold)
    
    img = val_dset[idx]['image'].permute(1, 2, 0)
    npi = np.array(img.numpy(), dtype = np.uint8)
    res0_int = int(res[0] * img.shape[1])
    gold0_int = int(gold[0] * img.shape[1])
    res1_int = int(res[1] * img.shape[1])
    gold1_int = int(gold[1] * img.shape[1])
    
    cv2.line(npi, (res0_int, 20), (res0_int, 420), (0, 0, 255), 2)
    cv2.line(npi, (res1_int, 20), (res1_int, 420), (0, 0, 128), 2)
    cv2.line(npi, (gold0_int, 20), (gold0_int, 420), (0, 255, 0), 2)
    cv2.line(npi, (gold1_int, 20), (gold1_int, 420), (0, 128, 0), 2)
    plt.figure(figsize=(20, 20))
    plt.imshow(npi)
    
    
    
    
    

In [ ]:

ctr = 0
for idx in range(len(train_dset)):

    res = model_ft(train_dset[idx]['image'].unsqueeze(0).to(device))[0].detach().cpu().numpy()
    print(res)
    gold = train_dset[idx]['bbox'].numpy() 
    print(gold)
    ctr += 1
    img = train_dset[idx]['image'].permute(1, 2, 0)
    npi = np.array(img.numpy(), dtype = np.uint8)
    res0_int = int(res[0] * img.shape[1])
    gold0_int = int(gold[0] * img.shape[1])
    res1_int = int(res[1] * img.shape[1])
    gold1_int = int(gold[1] * img.shape[1])
    
    cv2.line(npi, (res0_int, 20), (res0_int, 420), (0, 0, 255), 2)
    cv2.line(npi, (res1_int, 20), (res1_int, 420), (0, 0, 128), 2)
    cv2.line(npi, (gold0_int, 20), (gold0_int, 420), (0, 255, 0), 2)
    cv2.line(npi, (gold1_int, 20), (gold1_int, 420), (0, 128, 0), 2)
    plt.figure(figsize=(20, 20))
    plt.imshow(npi)
    
    if ctr > 10:
        break


In [ ]:
}